<a href="https://colab.research.google.com/github/nainye/nlp_practice/blob/main/word_embedding_gensim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 한글폰트 설치

In [ ]:
# 폰트 설치
!apt-get update -qq # 나눔고딕 설치
!apt-get install fonts-nanum* -qq

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 폰트 로딩
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

# Install

In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 17.3MB/s 


# Evn

In [ ]:
import os
import random
import shutil
import json
import zipfile
import math
import copy
import collections
import re

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sentencepiece as spm
import tensorflow as tf
import tensorflow.keras.backend as K

from tqdm.notebook import tqdm

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# random seed initialize
random_seed = 1234
random.seed(random_seed)
np.random.seed(random_seed)
tf.random.set_seed(random_seed)

In [ ]:
!nvidia-smi

Wed Jan 27 15:41:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Gensim
- https://radimrehurek.com/gensim/index.html

In [ ]:
import gensim
import gensim.downloader as api

## 한국어 학습

- https://wikidocs.net/50739

In [ ]:
# 행태소분석기 설치
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

In [ ]:
import konlpy
okt = konlpy.tag.Okt()

In [ ]:
okt.morphs("아버지가방에들어가신다")

['아버지', '가방', '에', '들어가신다']

In [ ]:
# 네이버 영화 리뷰 데이터 다운로드
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt

--2021-01-27 15:43:05--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19515078 (19M) [text/plain]
Saving to: ‘ratings.txt’

ratings.txt         100%[===================>]  18.61M  --.-KB/s    in 0.1s    

2021-01-27 15:43:05 (139 MB/s) - ‘ratings.txt’ saved [19515078/19515078]



In [ ]:
nsmc_data = pd.read_csv("ratings.txt", header=0, delimiter='\t', quoting=3)
print(f"전체 데이터의 개수: {len(nsmc_data)}")
nsmc_data.head(10)

전체 데이터의 개수: 200000


,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
5,2190435,사랑을 해본사람이라면 처음부터 끝까지 웃을수 있는영화,1
6,9279041,완전 감동입니다 다시봐도 감동,1
7,7865729,개들의 전쟁2 나오나요? 나오면 1빠로 보고 싶음,1
8,7477618,굿,1
9,9250537,바보가 아니라 병 쉰 인듯,1


In [ ]:
# null 제거
nsmc_data = nsmc_data.dropna()
print(f"null 제거 후 데이터의 개수: {len(nsmc_data)}")
nsmc_data.head(10)

null 제거 후 데이터의 개수: 199992


,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
5,2190435,사랑을 해본사람이라면 처음부터 끝까지 웃을수 있는영화,1
6,9279041,완전 감동입니다 다시봐도 감동,1
7,7865729,개들의 전쟁2 나오나요? 나오면 1빠로 보고 싶음,1
8,7477618,굿,1
9,9250537,바보가 아니라 병 쉰 인듯,1


In [ ]:
# 한글 이외의 문자 제거
nsmc_data['document'] = nsmc_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
nsmc_data.dropna()
print(f"한글 아닌 문자 제거 후 데이터의 개수: {len(nsmc_data)}")
nsmc_data.head(10)

한글 아닌 문자 제거 후 데이터의 개수: 199992


,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,1
2,4655635,폴리스스토리 시리즈는 부터 뉴까지 버릴께 하나도 없음 최고,1
3,9251303,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1
5,2190435,사랑을 해본사람이라면 처음부터 끝까지 웃을수 있는영화,1
6,9279041,완전 감동입니다 다시봐도 감동,1
7,7865729,개들의 전쟁 나오나요 나오면 빠로 보고 싶음,1
8,7477618,굿,1
9,9250537,바보가 아니라 병 쉰 인듯,1


In [ ]:
# 불용어 정의 (빈도가 너무 많은 단어는 학습에서 제외 함)
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [ ]:
# okt 형태소 분석기를 이용해 형태소 단위로 분할 (이때 불용어 제거)
tokens = []
for i, document in enumerate(tqdm(nsmc_data['document'], total=len(nsmc_data))):
    line = []
    line = okt.morphs(document)
    line = [word for word in line if not word in stopwords]
    tokens.append(line)

In [ ]:
print(len(tokens))
tokens[:10]

199992


[['어릴', '때', '보고', '지금', '다시', '봐도', '재밌어요', 'ㅋㅋ'],
 ['디자인',
  '을',
  '배우는',
  '학생',
  '외국',
  '디자이너',
  '그',
  '일군',
  '전통',
  '을',
  '통해',
  '발전',
  '해가는',
  '문화',
  '산업',
  '부러웠는데',
  '사실',
  '우리나라',
  '에서도',
  '그',
  '어려운',
  '시절',
  '끝',
  '까지',
  '열정',
  '을',
  '지킨',
  '노라노',
  '같은',
  '전통',
  '있어',
  '저',
  '같은',
  '사람',
  '꿈',
  '을',
  '꾸고',
  '이뤄',
  '나갈',
  '수',
  '있다는',
  '것',
  '감사합니다'],
 ['폴리스스토리', '시리즈', '부터', '뉴', '까지', '버릴께', '하나', '없음', '최고'],
 ['연기',
  '진짜',
  '개',
  '쩔구나',
  '지루할거라고',
  '생각',
  '했는데',
  '몰입',
  '해서',
  '봤다',
  '그래',
  '이런게',
  '진짜',
  '영화',
  '지'],
 ['안개', '자욱한', '밤하늘', '떠', '있는', '초승달', '같은', '영화'],
 ['사랑', '을', '해본', '사람', '라면', '처음', '부터', '끝', '까지', '웃을수', '있는', '영화'],
 ['완전', '감동', '입니다', '다시', '봐도', '감동'],
 ['개', '전쟁', '나오나요', '나오면', '빠', '로', '보고', '싶음'],
 ['굿'],
 ['바보', '아니라', '병', '쉰', '인듯']]

In [ ]:
# gensim 학습
word2vec_100 = gensim.models.Word2Vec(sentences=tokens, size=100, window=5, min_count=5)

In [ ]:
words = list(word2vec_100.wv.vocab)
len(words), words[:100]

(26174,
 ['어릴',
  '때',
  '보고',
  '지금',
  '다시',
  '봐도',
  '재밌어요',
  'ㅋㅋ',
  '디자인',
  '을',
  '배우는',
  '학생',
  '외국',
  '디자이너',
  '그',
  '전통',
  '통해',
  '발전',
  '해가는',
  '문화',
  '산업',
  '사실',
  '우리나라',
  '에서도',
  '어려운',
  '시절',
  '끝',
  '까지',
  '열정',
  '지킨',
  '같은',
  '있어',
  '저',
  '사람',
  '꿈',
  '꾸고',
  '이뤄',
  '나갈',
  '수',
  '있다는',
  '것',
  '감사합니다',
  '폴리스스토리',
  '시리즈',
  '부터',
  '뉴',
  '하나',
  '없음',
  '최고',
  '연기',
  '진짜',
  '개',
  '생각',
  '했는데',
  '몰입',
  '해서',
  '봤다',
  '그래',
  '이런게',
  '영화',
  '지',
  '안개',
  '밤하늘',
  '떠',
  '있는',
  '사랑',
  '해본',
  '라면',
  '처음',
  '웃을수',
  '완전',
  '감동',
  '입니다',
  '전쟁',
  '나오나요',
  '나오면',
  '빠',
  '로',
  '싶음',
  '굿',
  '바보',
  '아니라',
  '병',
  '쉰',
  '인듯',
  '내',
  '나이',
  '본',
  '나',
  '적',
  '이다',
  '하지만',
  '훗날',
  '사하나',
  '하',
  '감정',
  '완벽하게',
  '이해',
  '같다',
  '재밌다'])

In [ ]:
similar = word2vec_100.wv.most_similar("영화")
similar

[('애니', 0.613872766494751),
 ('다큐', 0.6115419864654541),
 ('애니메이션', 0.5967451333999634),
 ('공포영화', 0.5940037965774536),
 ('액션영화', 0.5865364074707031),
 ('작품', 0.5448018312454224),
 ('독립영화', 0.5418421030044556),
 ('만화영화', 0.5360821485519409),
 ('다큐멘터리', 0.5273967981338501),
 ('수작', 0.5263684988021851)]

In [ ]:
similar = word2vec_100.wv.most_similar("최민수")
similar

[('김희선', 0.9326217770576477),
 ('안성기', 0.9166045188903809),
 ('김승우', 0.9121912717819214),
 ('주진모', 0.9099200963973999),
 ('차승원', 0.9077414870262146),
 ('다우니', 0.9044102430343628),
 ('주니어', 0.9043997526168823),
 ('고소영', 0.9035391807556152),
 ('메릴', 0.9026621580123901),
 ('한석규', 0.9024266004562378)]

In [ ]:
similar = word2vec_100.wv.most_similar("장동건")
similar

[('에릭', 0.8947008848190308),
 ('안성기', 0.8936293721199036),
 ('이천희', 0.8934798836708069),
 ('윤여정', 0.8896331787109375),
 ('송은채', 0.8887768983840942),
 ('찌찌', 0.8822858333587646),
 ('이영애', 0.8801983594894409),
 ('한고은', 0.8781827688217163),
 ('장만옥', 0.8769034147262573),
 ('아델', 0.8765411972999573)]

In [ ]:
# 설경구 - 송윤아 + 고소영
result = word2vec_100.wv.most_similar(positive=['고소영', '설경구'], negative=['송윤아'])
result

[('혼신', 0.862230658531189),
 ('송강호', 0.8248682022094727),
 ('김혜수', 0.8198894262313843),
 ('차승원', 0.8158605098724365),
 ('최민수', 0.8138892650604248),
 ('하정우', 0.813798725605011),
 ('엄정화', 0.810836911201477),
 ('박중훈', 0.8108094334602356),
 ('한석규', 0.7945865988731384),
 ('디카프리오', 0.7931475639343262)]